**Import of the excel files**

In [5]:
# import of files and functions

import pandas as pd 
import numpy as np

# loading data
pilot_results = pd.read_excel('/Users/Ilaria/Library/CloudStorage/GoogleDrive-ilaria.mirlisenna@unitn.it/Il mio Drive/Projects/Uncertainty and Pain/Daniele/excel per analisi/RisultatiPilot.xlsx')
exp_results = pd.read_excel('/Users/Ilaria/Desktop/backup uncertainty /0707/risultatiparte2_corretto.xlsx')

actors_info = pd.read_excel('/Users/Ilaria/Library/CloudStorage/GoogleDrive-ilaria.mirlisenna@unitn.it/Il mio Drive/Projects/Uncertainty and Pain/Daniele/excel per analisi/analisiTtest_1706.xlsx', usecols = "A:C")


In [7]:
# checking that the correct files have been loaded

#for exp_results we use head 
exp_results.head()

#for pilot_results we try with the whole doc
pilot_results

# we try also actors_info
actors_info.head()

,SUBJ,Median,std
0,subject 1,NaN,NaN
1,HPP female8,63.0,24.0
2,HRP female8,71.0,30.0
3,subject 2,NaN,NaN
4,HPP female13,63.0,18.0


In [42]:
# calculating the median values for the measures of interest for each video

# first, i identify all the videos
videos = pilot_results['Video'].unique()

# i initialize an empty list where i am gonna put the measures of interest 
table_median = []

for video in videos:
    curr_video = pilot_results[pilot_results['Video'] == video]
    table_median.append({
        'Video': video.replace('.mp4', ''),
        'MedianPainFinal': np.median(curr_video['PainFinal']),
        'MedianRealFinal': np.median(curr_video['RealFinal'])})

# i create a dataframe out of the list
table_median = pd.DataFrame(table_median)
table_median_sorted =  table_median.sort_values(by='Video')

# i rename the videos to match the names in the results doc
table_median['Video'] = table_median['Video'].str.replace('Males ', 'Males')
table_median['Video'] = table_median['Video'].str.replace('female ', 'female')


## Building an Actor-Video table

Here, we create a table that links each actor to their corresponding posed and real videos.
Starting from the original "actors_info" dataset, we loop through the table and whenever we find a row containing "subject", we take the next two rows as the posed and real videos.
Next, we create an organized and clean table that contains the columns "Actor", "Posed_Video" and "Real_Video", which allows us to easily identify which actor performed in each video used during the experiment.

In [14]:
# linking the video names to the actor who performed them

# i initialize an empty list
temporary_rows = []

# I create an organized table that contains for each row the name of the actor and the corresponding posed and real videos
for i in range(len(actors_info) - 2):
    if "subject" in str(actors_info.loc[i, "SUBJ"]):
        curr_subj = actors_info.loc[i, "SUBJ"]
        curr_posed = actors_info.loc[i + 1, "SUBJ"]
        curr_real = actors_info.loc[i + 2, "SUBJ"]
        
        temporary_rows.append({"Actor": curr_subj, "Posed_Video": curr_posed, "Real_Video": curr_real})

t_actor_names = pd.DataFrame(temporary_rows)

# checking if everything is ok
t_actor_names


,Actor,Posed_Video,Real_Video
0,subject 1,HPP female8,HRP female8
1,subject 2,HPP female13,HRP female12
2,subject 3,HPP female10,HRP female11
3,subject 5,HPP female4,HRP female5
4,subject 6,HPP female1,HRP female1
5,subject 7,HPP female3,HRP female4
6,subject 8,LPP female6,LRP female7
7,subject 9,HPP female14,HRP female13
8,subject 10,HPP female9,HRP female9
9,subject 11,HPP female7,HRP female7


## Building the comprehensive experimental database for the analyses

In this section, we combine the different information to obtain a comprehensive dataset with all the information needed for the analyses. 

Steps include:
1. Initializing an empty dataframe and adding core experimental information
2. Adding and deriving information about the Actor performing each video
3. Adding additional experimental information to be used as predictors or dependant variables during later analyses models

In [50]:

# initialize an empty database for comprehensive analyses
database = pd.DataFrame()

# print the experimental results dataset to keep track of the information to add
print(exp_results.head())

# add core information from the experimental results dataset
database['Subject_ID'] = exp_results['subjID']
database['Trial'] = exp_results['trial']
database['SubjGender'] = exp_results['gender']

# initialize an empty column for storing the actor performing each video
database['Actor'] = None

# link each video in exp_results to the corresponding actor
for i, row in exp_results.iterrows():
    curr_video = row['video']
    is_match = (t_actor_names[['Posed_Video', 'Real_Video']] == curr_video).any(axis = 1)
    curr_actor = t_actor_names.loc[is_match, 'Actor']

    if not curr_actor.empty:
            exp_results.loc[i, 'Actor'] = curr_actor.iloc[0]
    else:
            exp_results.loc[i, 'Actor'] = None

# add information about the actor and the video        
database['Actor'] = exp_results['Actor']
database['Face_ID'] = exp_results['video']

# derive the gender of the actor performing the video
database['Actor_Gender'] = np.where(database['Face_ID'].str.contains('female'), 'F', np.where(database['Face_ID'].str.contains('Males'), 'M', None))

# add additional information from the experimental results dataset
database['Simulation'] = exp_results['Simulation']
database['MU_VideoPain'] = exp_results['pilot median pain']

# assign median value for the Reality scores 
mu_reality_list = []

for i, row in exp_results.iterrows():
    curr_video = row['video']
    is_match = table_median[table_median['Video'] == curr_video.replace('.mp4', '')]
    if not is_match.empty:
        mu_reality_list.append(is_match['MedianRealFinal'].values[0])
    else:
        mu_reality_list.append(np.nan)


database['MU_VideoReality'] = mu_reality_list

# add missing information 
database['Target_cue_mean'] = exp_results['mu target']
database['Actual_cue_mean'] = exp_results['dots median']
database['Uncertainty'] = exp_results['Uncertainty']
database['PainFinal'] = exp_results['Painfinal']
database['Pain_RT'] = exp_results['Paintime']
database['Pain_Path'] = exp_results['Painfinal'] - exp_results['painInitial']
database['VerFinal'] = exp_results['Realfinal']
database['Ver_rt'] = exp_results['Realtime']
database['Ver_Path'] = exp_results['Realfinal'] - exp_results['Realinitial']


<class 'list'>
